In [ ]:
# !pip install easyocr
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128
# !pip install ultralytics

In [6]:
import easyocr

path = "E:\Github_repository\license-plate-project\image\image01.jpg"
reader = easyocr.Reader(['th', 'en'], gpu=True)
reader.readtext(path, detail=0)

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
e:\myworkspace\.venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


['กท', '2058', 'ฉะเชิงเทรา', 'ป6', '572982949']

In [ ]:
import cv2 as cv
from ultralytics import YOLO
# Open camera
cam = cv.VideoCapture(0)
cam.set(cv.CAP_PROP_FRAME_WIDTH, 1280)  # Set width to 1280
cam.set(cv.CAP_PROP_FRAME_HEIGHT, 720)  # Set height to 720

# Load YOLO model
model = YOLO('yolo11_custom.pt') 



try:
    while True:
        success, img = cam.read()
        if not success:
            break

        results = model(img, stream=True)

        for r in results:
            boxes = r.boxes
            for box in boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                w, h = x2 - x1, y2 - y1
                conf = math.ceil((box.conf[0] * 100)) / 100
                cls = int(box.cls[0])

                if conf >= 0.80:
                    cropped = img[y1:y2, x1:x2]
                    gray = cv.cvtColor(cropped, cv.COLOR_BGR2GRAY)
                    _, thresh = cv.threshold(gray, 127, 255, cv.THRESH_BINARY)
                    
                    ocr_text = pytesseract.image_to_string(thresh, lang='eng+jpn')
                    print("[OCR]:", ocr_text)

                    yen_price = extract_yen_price(ocr_text)
                    if currency_choice in data_currency: # Check if the currency is in the data_currency dictionary
                        EXCHANGE_RATE = data_currency[currency_choice] 
                        symbol = currency_choice
                    else: 
                        print("someting whent wrong")
                    
                    if yen_price and currency_choice is not None:
                        currency_price = round(yen_price * EXCHANGE_RATE, 2)
                        label_text = f'YEN{yen_price} / {symbol} {currency_price}'
                    else:
                        label_text = "Price not found"

                    cvz.cornerRect(img, (x1, y1, w, h))
                    cvz.putTextRect(img, label_text, (x1, y1 - 10), scale=1, thickness=1, colorR=(150, 0, 255), colorT=(0, 0, 0), offset=10)

        cv.imshow("YOLO + OCR", img)
        cv.setWindowProperty("YOLO + OCR", cv.WND_PROP_FULLSCREEN, cv.WINDOW_FULLSCREEN)
        if cv.waitKey(1) & 0xFF == ord('q'):
            break
except KeyboardInterrupt:
    print("Exiting program...")